# 1

经典求排列组合

逐位判断可能的取值数量，代码不够简洁。


In [ ]:
from typing import List

class Solution:
    def atMostNGivenDigitSet(self, digits: List[str], num: int) -> int:
        # digits = list(map(int, digits))
        digits = list(map(int, digits))
        num_list = list(map(int, list(str(num))))
        count = 0
        n1 = len(digits)    # 可选择数字个数
        n2 = len(str(num))    # 目标值的位数

        # 1.处理最高位的情况
        def find_min(digits, target):   # 找到小于等于目标值的位置(digits有序) (替换为字典更舒服)
            l, r = 0, len(digits)-1
            while l < r:
                m = (l + r) // 2
                if digits[m] == target:
                    return m + 1    # 索引值少-1
                elif digits[m] > target:
                    r = m - 1
                elif digits[m] < target:
                    l = m + 1
            l = l if digits[l] > target else l + 1
            return l
        
        # 1.1 判断是否存在位数一样 且小于目标值的数
        is_all_usable = True    # num的每一位都能找到对应的数
        for x in num_list:
            if digits[find_min(digits, x) - 1] != x:
                is_all_usable = False
                break
        is_max_usable = find_min(digits, num_list[0]) # 最高位能找到数

        # 1.2 如果存在，可以计算小于目标值的数量
        if is_all_usable:   # 每一位都可找到
            count += 1
        if is_max_usable:   # 最大的一位能找到
            num_size = len(num_list)
            digits_size = len(digits)
            prob = 0
            for i in range(num_size):    # 不同位的可能性相乘
                x = num_list[i]
                idx = find_min(digits, x)
                idx_copy = idx
                idx = max(0, idx - 1 if digits[idx-1] == x else idx)
                prob += idx * digits_size ** (num_size - 1 - i)
                if digits[idx_copy - 1] != x:   # 当前位不能在digital中找到时，下一位没必要再循环，因为都被统计在内了
                    break
            count += prob

        # 2.n少一位的情况
        for nn2 in range(1, n2):    # 解决高位为'0'的情况
            count += n1 ** (nn2)
        return count

执行用时：28 ms, 在所有 Python3 提交中击败了97.01%的用户

内存消耗：15 MB, 在所有 Python3 提交中击败了49.25%的用户

time: O(n)

space: O(1)

（审题后，数字是1-9之间，没有0，上述代码存在多余部分。）

# 题解

方法一：数位动态规划

将问题分为2部分： \
第1部分，与目标数的位数少于`k < K`的合法值有 D^k个。 \
第2部分，与目标数的位数相等时，又分情况。第1位小于目标的第1位，合法值有 D^(k-1)个；第1位等于目标的第1位，那么考虑第2位，以此类推。

In [9]:
class Solution:
    def atMostNGivenDigitSet(self, D, N):
        S = str(N)
        K = len(S)
        dp = [0] * K + [1]
        # dp[i] = total number of valid integers if N was "N[i:]"

        for i in range(K-1, -1, -1):
            # Compute dp[i]

            for d in D:
                if d < S[i]:        # 当前位i的值d小于S[i]时，合法取值的数量为K-i-1个位的所有可能。
                    dp[i] += len(D) ** (K-i-1)
                elif d == S[i]:     # 当前位i的值相等S[i]时，合法取值的数量为i+1位的数量。
                    dp[i] += dp[i+1]    # dp的i会依赖i+1。          2.dp[0]最终的结果是目标数N的位数==K的所有取值可能。
        return dp[0] + sum(len(D) ** i for i in range(1, K))    # 1.后半部分是目标数N的位数<K的所有取值可能。


if __name__ == '__main__':
    D_list = [
        ['2','3','4','5'],
        ['1', '3', '5', '7'],
    ]
    N_list = [2345, 57]

    for D, N in zip(D_list, N_list):
        print(Solution().atMostNGivenDigitSet(D, N))

112
16


方法二：数学

令 B = |D|，一个合法的数仅包含 D 中的数字，如果我们把 D 中数字从小到大映射为 [1 .. B]，那么将对我们的计数有很大的便利。例如，当 D 包含 [1, 3, 5, 7] 时，我们将它映射为 [1, 2, 3, 4]，那么合法的数也从 1, 3, 5, 7, 11, 13, 15, 17, 31, ... 映射为 1, 2, 3, 4, 11, 12, 13, 14, 21, ...。这样的好处是，对于任何一个映射好的数，我们可以用类似进制转换的方式，得到它是第一个合法的数，例如 57(/58/59)->34 就是第 3 * B + 4 = 3 * 4 + 4 = 16 个合法的数。

注： :)想过将数字映射到1~B，但没想到有这样的关系。

In [10]:
import bisect

class Solution(object):
    def atMostNGivenDigitSet(self, D, N):
        B = len(D) # bijective-base B
        S = str(N)
        K = len(S)
        A = []  #  The largest valid number in bijective-base-B.

        # 1.映射
        for c in S:
            if c in D:
                A.append(D.index(c) + 1)
            else:
                i = bisect.bisect(D, c)
                A.append(i)
                # i = 1 + (largest index j with c >= D[j], or -1 if impossible)
                if i == 0:
                    # subtract 1
                    for j in range(len(A) - 1, 0, -1):
                        if A[j]: break
                        A[j] += B
                        A[j-1] -= 1

                A.extend([B] * (K - len(A)))
                break

        # 2.计算
        ans = 0
        for x in A:
            ans = ans * B + x   # K==4时，A[0]*4^3 + A[1]*4^2 + A[2]*4^1 + A[3]
        return ans


if __name__ == '__main__':
    D_list = [
        ['2','3','4','5'],
        ['1', '3', '5', '7'],
    ]
    N_list = [2345, 57]

    for D, N in zip(D_list, N_list):
        print(Solution().atMostNGivenDigitSet(D, N))

[1, 2, 3, 4]
112
[3, 4]
16
